# 对象引用、可变性和垃圾回收

## 8.1变量不是盒子

In [1]:
a = [1, 2, 3]
b = a
a.append(4)
b

[1, 2, 3, 4]

In [2]:
class Gizmo:
    def __init__(self):
        print('Gizmo id: %d' % id(self))

In [3]:
x = Gizmo()

Gizmo id: 4500322640


In [4]:
y = Gizmo() * 10

Gizmo id: 4500323920


TypeError: unsupported operand type(s) for *: 'Gizmo' and 'int'

In [6]:
dir() 
# dir() 函数不带参数时，返回当前范围内的变量、方法和定义的类型列表；带参数时，返回参数的属性、方法列表。如果参数包含方法__dir__()，该方法将被调用。如果参数不包含__dir__()，该方法将最大限度地收集参数信息

['Gizmo',
 'In',
 'Out',
 '_',
 '_1',
 '_5',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_i',
 '_i1',
 '_i2',
 '_i3',
 '_i4',
 '_i5',
 '_i6',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'a',
 'b',
 'exit',
 'get_ipython',
 'quit',
 'x']

## 8.2 标识、相等性和别名

In [7]:
charles = {'name': 'Charles L. Dodgson', 'born': 1832}
lewis = charles

In [8]:
lewis is charles

True

In [9]:
id(charles), id(lewis)

(4500278800, 4500278800)

In [10]:
alex = {'name': 'Charles L. Dodgson', 'born': 1832}

In [11]:
alex == charles

True

In [12]:
alex is not charles

True

In [13]:
id(charles), id(alex)

(4500278800, 4500216960)

### 8.2.2　元组的相对不可变性

In [14]:
t1 = (1, 2, [30, 40])
t2 = (1, 2, [30, 40])

In [15]:
t1 == t2

True

In [16]:
id(t1[-1])

4499253520

In [17]:
t1[-1].append(99)

In [18]:
id(t1[-1])

4499253520

In [19]:
t1

(1, 2, [30, 40, 99])

## 8.3　默认做浅复制

In [20]:
l1 = [3, [55, 44], (7, 8, 9)]
l2 = list(l1)

In [21]:
l1,l2

([3, [55, 44], (7, 8, 9)], [3, [55, 44], (7, 8, 9)])

In [22]:
l1 == l2

True

In [23]:
l1 is l2,id(l1),id(l2)

(False, 4509394016, 4509395776)

In [25]:
# http://www.pythontutor.com/
l1 = [3, [66, 55, 44], (7, 8, 9)]
l2 = list(l1) # 这里和l1[:]等效
l1.append(100) 
l1[1].remove(55) 
print('l1:', l1) 
print('l2:', l2)
l2[1] += [33, 22] 
print(id(l2[2]))
l2[2] += (10, 11)
print(id(l2[2]))
print('l1:', l1) 
print('l2:', l2)

l1: [3, [66, 44], (7, 8, 9), 100]
l2: [3, [66, 44], (7, 8, 9)]
4499845520
4499887216
l1: [3, [66, 44, 33, 22], (7, 8, 9), 100]
l2: [3, [66, 44, 33, 22], (7, 8, 9, 10, 11)]


### 为任意对象做深复制和浅复制

In [30]:
class Bus:
    
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            print(id(passengers))
            self.passengers = list(passengers)
            print(id(self.passengers))

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

In [31]:
import copy
bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1) # 浅复制
bus3 = copy.deepcopy(bus1) # 深复制
bus1.drop('Bill')

4509214832
4509427344


In [28]:
bus2.passengers

['Alice', 'Claire', 'David']

In [29]:
bus3.passengers

['Alice', 'Bill', 'Claire', 'David']

In [32]:
# 循环引用
a = [10, 20]
b = [a, 30]
a.append(b)

In [33]:
a,len(a)

([10, 20, [[...], 30]], 3)

In [36]:
id(a[2]),id(b)

(4499761664, 4499761664)

<iframe width="800" height="500" frameborder="0" src="http://pythontutor.com/iframe-embed.html#code=a%20%3D%20%5B10,%2020%5D%0Ab%20%3D%20%5Ba,%2030%5D%0Aa.append%28b%29&codeDivHeight=400&codeDivWidth=350&cumulative=false&curInstr=3&heapPrimitives=nevernest&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false"> </iframe>

In [37]:
from copy import deepcopy
c = deepcopy(a)
c

[10, 20, [[...], 30]]

## 8.4 函数的参数作为引用时

In [39]:
def f(a,b):
    a += b
    return a

In [40]:
x = 1
y = 2
f(x,y)

3

In [42]:
x = [1,2]
y = [3,4]
f(x,y)

[1, 2, 3, 4]

In [43]:
x,y

([1, 2, 3, 4], [3, 4])

In [44]:
t = (10,20)
u = (30,40)
f(t,u)

(10, 20, 30, 40)

In [45]:
t,u

((10, 20), (30, 40))

### 8.4.1　不要使用可变类型作为参数的默认值

In [46]:
class HauntedBus:
    """A bus model haunted by ghost passengers"""

    def __init__(self, passengers=[]):  # <1>
        self.passengers = passengers  # <2>

    def pick(self, name):
        self.passengers.append(name)  # <3>

    def drop(self, name):
        self.passengers.remove(name)

In [47]:
bus1 = HauntedBus(['Alice', 'Bill'])

In [48]:
bus1.passengers

['Alice', 'Bill']

In [49]:
bus1.pick('Charlie')
bus1.drop('Alice')
bus1.passengers

['Bill', 'Charlie']

In [52]:
bus2 = HauntedBus()
bus2.passengers

[]

In [53]:
bus2.pick('Carrie')
bus2.passengers

['Carrie']

In [54]:
bus3 = HauntedBus()
bus3.passengers

['Carrie']

In [55]:
bus3.pick('Dave')
bus2.passengers

['Carrie', 'Dave']

In [56]:
bus2.passengers is bus3.passengers

True

In [57]:
HauntedBus.__init__.__defaults__

(['Carrie', 'Dave'],)

### 8.4.2　防御可变参数

In [58]:
class TwilightBus:
    """A bus model that makes passengers vanish"""

    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = passengers

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

In [59]:
basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat'] 
bus = TwilightBus(basketball_team)
bus.drop('Tina') 
bus.drop('Pat')
basketball_team

['Sue', 'Maya', 'Diana']

## 8.5　del和垃圾回收

In [60]:
import weakref
s1 = {1, 2, 3}
s2 = s1
def bye():
    print('Gone with the wind')

In [61]:
ender = weakref.finalize(s1, bye)
ender.alive

True

In [62]:
del s1
ender.alive

True

In [63]:
s2 = 'hhh'
ender.alive

Gone with the wind...


False

## 8.6　弱引用

In [66]:
import weakref
a_set = {0, 1}
wref = weakref.ref(a_set) 
wref

<weakref at 0x10cc91350; to 'set' at 0x10cc8a230>

In [67]:
wref()

{0, 1}

In [68]:
a_set = {2,3,4}
wref()

{0, 1}

In [89]:
wref() is None

False

### 8.6.1　WeakValueDictionary简介

In [90]:
class Cheese:

    def __init__(self, kind):
        self.kind = kind

    def __repr__(self):
        return 'Cheese(%r)' % self.kind

In [91]:
import weakref
stock = weakref.WeakValueDictionary()
catalog = [Cheese('Red Leicester'), Cheese('Tilsit'),
                Cheese('Brie'), Cheese('Parmesan')]

for cheese in catalog:
    stock[cheese.kind] = cheese

In [92]:
sorted(stock.keys())

['Brie', 'Parmesan', 'Red Leicester', 'Tilsit']

In [95]:
del catalog 

['Parmesan']

In [100]:
sorted(stock.keys())

['Parmesan']

In [101]:
del cheese

In [102]:
sorted(stock.keys())

[]

### 8.6.2　弱引用的局限

## 8.7　Python对不可变类型施加的把戏

In [103]:
t1 = (1,2,3)
t2 = tuple(t1)
t3 = t1[:]

In [104]:
id(t1),id(t2),id(t3)

(4499847600, 4499847600, 4499847600)

In [105]:
t3 = (1,2,3)
t1 is t3

False

In [106]:
s1 = 'ABC'
s2 = 'ABC'
s1 is s2

True